In [1]:
import sys
import pandas as pd
import numpy as np
import nasdaqdatalink as ndl
import os
import psycopg2 as psg



In [2]:
ndl.ApiConfig.api_key = os.getenv('NASDAQ_DATA_LINK_API_KEY')
# try:
#fetching region Table
region_table = pd.DataFrame(ndl.get_table('ZILLOW/REGIONS', paginate = True))



# except Exception as e:
#     print(f"Error: {e}")




In [3]:

# region_table['region'].astype(str)
# region_table['region_type'].astype(str)

#filtering df for only CA regions
california_regionsdf = region_table[region_table['region'].str.contains('CA', na = False)]

#print(california_regionsdf)

#List of available regions
region_types = list(california_regionsdf.drop_duplicates(subset = ['region_type'])['region_type'])

#remove these as i do not want to track info for these region types
region_types.remove('neigh')
region_types.remove('metro')


print(region_types)



['city', 'zip', 'county']


In [15]:
#creating DataFrames for each city, county, zip#

#creating df for city
#creating copy of the data frame that only contains city types
full_city_df = california_regionsdf[california_regionsdf['region_type'].str.contains(region_types[0])].copy()

#print(full_city_df)

#adding the state and city section from the region info
full_city_df[['city','state','_']] = full_city_df['region'].str.split(';', n = 2, expand = True)

#city data frame that only contains needed data for user
city_df = full_city_df[['region_id','region_type', 'city', 'state']]

#print(city_df)



In [22]:
#creating data frame for zip

#create copy df of regions that are only zip info
full_zip_df = california_regionsdf[california_regionsdf['region_type'].str.contains(region_types[1])].copy()

#add zipcode and state columns to df
full_zip_df[['zipcode', 'state', '_']] = full_zip_df['region'].str.split(';', n = 2, expand = True)

#df to be pushed to db
zip_df = full_zip_df[['region_id', 'region_type', 'zipcode', 'state']]

#print(zip_df)

In [30]:
#creating df for county

full_county_df = california_regionsdf[california_regionsdf['region_type'].str.contains(region_types[2])].copy()

#add county and state columns to df
full_county_df[['county', 'state', '_']] = full_county_df['region'].str.split(';', n = 2, expand = True)

#df to be pushed to db
county_df = full_county_df[['region_id', 'region_type', 'county', 'state']]
#print(county_df)

In [31]:
indicator_table = ndl.get_table('ZILLOW/INDICATORS')

print(indicator_table)

     indicator_id                                          indicator  \
None                                                                   
0            ZSFH           ZHVI Single-Family Homes Time Series ($)   
1            ZCON                   ZHVI Condo/Co-op Time Series ($)   
2            ZATT           ZHVI All Homes- Top Tier Time Series ($)   
3            ZALL  ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)   
4            ZABT        ZHVI All Homes- Bottom Tier Time Series ($)   
5            Z5BR                    ZHVI 5+ Bedroom Time Series ($)   
6            Z4BR                     ZHVI 4-Bedroom Time Series ($)   
7            Z3BR                     ZHVI 3-Bedroom Time Series ($)   
8            Z2BR                     ZHVI 2-Bedroom Time Series ($)   
9            Z1BR                     ZHVI 1-Bedroom Time Series ($)   
10           SSSW  Median Sale Price (Smooth, SFR only, Weekly View)   
11           SSSM      Median Sale Price (Smooth, SFR only, Mont

In [ ]:
home_region_id = home_zip_info['region_id']

home_data_table = pd.DataFrame(ndl.get_table('ZILLOW/DATA', region_id = home_region_id))

home_data_values = home_data_table['value'].round(2)

print(home_data_values)